# Import & Load Abandonment Data

### Configuration

In [3]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

### Main Parameters

In [4]:
remote_path = ''
local_path = '../data_final/'

In [5]:
import postgres #from local file postgres.py
from commons import inline_table_xml, download_file, download_zip_file, extract_zip_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

### Load File

In [ ]:
#Open unzipped file
df = pd.read_csv(local_path + 'mds_abandonment_rate.csv', delimiter = ",")
df = df[['comuna_id', 'year', 'education_id', 'promotion_id', 'total']]
df = df.astype({'comuna_id':'int', 'year':'int', 'education_id':'int', 'promotion_id':'int', 'total':'int'})
list(df)

### Ingest

In [ ]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, 'childhood', 'fact_abandonment')

### Add Indices to Foreign Keys

In [ ]:
engine.execute("""
CREATE INDEX fact_abandonment_comuna_id_index 
ON childhood.fact_abandonment (comuna_id)
""")

engine.execute("""
CREATE INDEX fact_abandonment_year_index 
ON childhood.fact_abandonment ("year");
""")

### Relation tables

In [6]:
d2 = download_file(remote_path, local_path, 'promotion.csv')
print(inline_table_xml(d2, 'name_en', 'id', 'name_es'))

Already downloaded. Using: ../data_final/promotion.csv

<InlineTable alias="name_en">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">1</Value>
      <Value column="description">Aprobados</Value>
      <Value column="es_description">Aprobados</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">Reprobados</Value>
      <Value column="es_description">Reprobados</Value>
    </Row>
  <Row>
      <Value column="id">3</Value>
      <Value column="description">Abandonos</Value>
      <Value column="es_description">Abandonos</Value>
    </Row>
  </Rows>
</InlineTable>
    
